<a href="https://colab.research.google.com/github/GenAIHub/genai-workshop/blob/main/03_Agents/02_simple_multistep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-Step Query Engine

We have a multi-step query engine that's able to decompose a complex query into sequential subquestions.
This lab walks you through how to set it up!

In [ ]:
!pip install boto3
!pip install llama-index
!pip install llama-index-llms-bedrock
!pip install llama-index-embeddings-bedrock

#### Download Data

In [ ]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

#### Load documents, build the VectorStoreIndex

In [3]:
import os
import boto3

# Set AWS env config
region_name = os.getenv("AWS_REGION", "us-east-1")
aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID", "")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY", "")

In [4]:
from llama_index.llms.bedrock import Bedrock
from llama_index.embeddings.bedrock import BedrockEmbedding
from llama_index.core import Settings

llm_model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
embed_model_id = "amazon.titan-embed-text-v1"

llm = Bedrock(
    model=llm_model_id,
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=region_name,
    temperature=0.1,
    max_tokens=512
    )

embed_model = BedrockEmbedding(
    model=embed_model_id,
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=region_name,
    )

Settings.llm = llm
Settings.embed_model = embed_model

In [5]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from IPython.display import Markdown, display

In [6]:
# load documents
documents = SimpleDirectoryReader("./data/paul_graham/").load_data()

In [7]:
index = VectorStoreIndex.from_documents(documents)

#### Query Index

In [8]:
from llama_index.core.indices.query.query_transform.base import (
    StepDecomposeQueryTransform,
)

query_engine = index.as_query_engine(llm=llm, similarity_top_k=4)
step_decompose_transform = StepDecomposeQueryTransform(llm=llm, verbose=True)

In [9]:
index_summary = "Used to answer questions about the author"

In [10]:
# set Logging to DEBUG for more detailed outputs
from llama_index.core.query_engine import MultiStepQueryEngine


query_engine = MultiStepQueryEngine(
    query_engine=query_engine,
    query_transform=step_decompose_transform,
    index_summary=index_summary,
    num_steps=4
)
response = query_engine.query(
    "Who was in the first batch of the accelerator program the author started?",
)

> Current query: Who was in the first batch of the accelerator program the author started?
> New query: Given the context "Used to answer questions about the author" and no previous reasoning steps, the next relevant question could be:

"Did the author start an accelerator program?"

This question directly relates to the original query about who was in the first batch of an accelerator program started by the author. By confirming if the author actually started such a program, we can then potentially ask follow-up questions about the participants if the context provides that information.
> Current query: Who was in the first batch of the accelerator program the author started?
> New query: Based on the context provided and the previous reasoning steps, a relevant follow-up question could be:

"Who were the startups/founders in the first batch of Y Combinator in 2005?"

This directly addresses the original question of who was in the first batch of the accelerator program started by the a

In [13]:
display(Markdown(f"<b>{response}</b>"))

<b>Unfortunately, the provided context does not explicitly list out the names of all 8 startup groups that were funded in the very first batch of Y Combinator's Summer Founders Program in 2005. 

The context mentions that notable companies/founders like Reddit, Twitch (founded by Justin Kan and Emmett Shear), Aaron Swartz who helped create RSS, and Sam Altman who later became Y Combinator's president, were part of that inaugural 2005 batch. 

However, it does not provide the full list of names for all 8 startups funded in that pioneering first accelerator cohort started by author Paul Graham. Without those specific details in the given context, I cannot comprehensively answer who the complete set of companies/founders were in that first Y Combinator batch.</b>

In [14]:
sub_qa = response.metadata["sub_qa"]
tuples = [(t[0], t[1].response) for t in sub_qa]
print(tuples)

[('Given the context "Used to answer questions about the author" and no previous reasoning steps, the next relevant question could be:\n\n"Did the author start an accelerator program?"\n\nThis question directly relates to the original query about who was in the first batch of an accelerator program started by the author. By confirming if the author actually started such a program, we can then potentially ask follow-up questions about the participants if the context provides that information.', 'Yes, based on the context information provided, the author Paul Graham did start an accelerator program called Y Combinator (YC). The relevant details are:\n\n- In March 2005, Graham, his wife Jessica Livingston, and friends Robert Morris and Trevor Blackwell decided to start their own investment firm called Y Combinator to fund and advise startups in batches.\n\n- In the summer of 2005, they ran the first batch called the "Summer Founders Program" where they funded 8 startup groups, including t